In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip install GPT4All

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 41.2 MB/s eta 0:00:00


In [ ]:
from gpt4all import GPT4All   # take just a string


we will use gpt4all model insted of openAI gpt model.

test our LLM

In [ ]:
# download the model weights
llm = GPT4All("ggml-gpt4all-l13b-snoozy.bin")

100%|██████████| 8.14G/8.14G [03:13<00:00, 42.1MiB/s]


Model downloaded at:  /root/.cache/gpt4all/ggml-gpt4all-l13b-snoozy.bin


In [ ]:
output = llm.generate("The capital of France is ", max_tokens=3)
print(output)


 Paris.


In [ ]:
! pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1


In [ ]:
from langchain.llms import GPT4All


In [ ]:
# langchain model takes the path of the weights
llm = GPT4All(model = "/root/.cache/gpt4all/ggml-gpt4all-l13b-snoozy")

Found model file at  /root/.cache/gpt4all/ggml-gpt4all-l13b-snoozy.bin


load out data which is consist of two CSV files.

the first contains movies information.


the second contains users information.


In [ ]:
import pandas as pd
import numpy as np


In [45]:

movies = pd.read_csv('/content/gdrive/MyDrive/GPT_Recommendation_Sysytem/ml-25m/movies.csv')
print(movies.shape)
movies

(62423, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [46]:
ratings = pd.read_csv('/content/gdrive/MyDrive/GPT_Recommendation_Sysytem/ml-25m/ratings.csv')
print(ratings.shape)
ratings

(1048575, 4)


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
1048570,7045,4447,3.5,1164258032
1048571,7045,4720,4.0,1164257756
1048572,7045,4857,4.0,1164242753
1048573,7045,4886,5.0,1168033506


Now we have users and movies data

to recommend a movie for a specific user you need to separate the movies that the user had watched and the movies that he had not watched.

after that we will take some of "watched_movies" and some of "unwatched_movies" to pass them to our LLM.


Lets do that for user 1

In [ ]:
user = 1


In [ ]:
def user_movies(user):
  watched_movies = ratings[ratings['userId']==user]
  watched_movies['title'],watched_movies['genres']=ratings['movieId'],ratings['movieId']

  idx =[]
  for i,id in enumerate(watched_movies['movieId']):
    idx.extend(movies.index[movies['movieId']==id])
    watched_movies['title'][i] = movies['title'][idx[-1]]
    watched_movies['genres'][i] = movies['genres'][idx[-1]]

  unwatched_movies = movies.drop(idx)

  return watched_movies,unwatched_movies

watched_movies,unwatched_movies = user_movies(user)



<ipython-input-12-1822fbbccc95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watched_movies['title'],watched_movies['genres']=ratings['movieId'],ratings['movieId']
<ipython-input-12-1822fbbccc95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watched_movies['title'],watched_movies['genres']=ratings['movieId'],ratings['movieId']
<ipython-input-12-1822fbbccc95>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

In [ ]:
print(watched_movies.shape)
watched_movies.drop(['userId','movieId','timestamp'],inplace=True,axis=1)
watched_movies.head()

(70, 6)


<ipython-input-13-6f2480d77a8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watched_movies.drop(['userId','movieId','timestamp'],inplace=True,axis=1)


,rating,title,genres
0,5.0,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,3.5,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,5.0,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,5.0,Underground (1995),Comedy|Drama|War
4,3.5,Singin' in the Rain (1952),Comedy|Musical|Romance


In [ ]:
print(unwatched_movies.shape)
unwatched_movies.head()

(62353, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy



We will not pass all the "watched_movies" to our LLM (reducing the input tokens) so lets choose 10 random movies.

5 with a rating larger than 3, and 5 less than 3 to make it easy for the LLM to understand which type of movies is the best for that user and which is the worst.

In [ ]:
import random
def get10random_movies():
  low_ratings  = watched_movies[watched_movies['rating'] <3]
  high_ratings = watched_movies[watched_movies['rating'] >3]
  random1= random.sample(list(range(low_ratings.shape[0])),5)
  random2= random.sample(list(range(high_ratings.shape[0])),5)
  low_ratings_movies  = low_ratings.iloc[random1]
  high_ratings_movies = high_ratings.iloc[random2]
  random_movies = pd.concat([low_ratings_movies,high_ratings_movies])
  random_movies = random_movies.sample(frac=1)
  return random_movies

random_movies = get10random_movies()
random_movies

,rating,title,genres
59,3.5,Hour of the Wolf (Vargtimmen) (1968),Drama|Horror
58,4.0,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...
46,3.5,"Good bye, Lenin! (2003)",Comedy|Drama
61,3.5,Swann in Love (Un amour de Swann) (1984),Drama
31,0.5,"Piano Teacher, The (La pianiste) (2001)",Drama
52,2.5,Winter Light (Nattvardsgästerna) (1963),Drama
50,2.5,"Virgin Spring, The (Jungfrukällan) (1960)",Crime|Drama
13,2.5,Back to the Future Part III (1990),Adventure|Comedy|Sci-Fi|Western
39,4.0,Finding Nemo (2003),Adventure|Animation|Children|Comedy
12,2.5,Back to the Future Part II (1989),Adventure|Comedy|Sci-Fi


We have more than 62000 "unwatched_movies" for any user. We can not pass all of them to our LLM.

What you will do in this problem??

We have 10 "watched_movies", lets get the most similar 10 movies for each one of the 10 "watched_movies" and we will get 100 "unwatched_movies" to pass them to our LLM.

But HOW??

It's so simple by Embedding, but first lets compine the movies names and genres(types) in one string, after that we pass it for Embedding.

embedding and vector store

In [ ]:
from langchain.embeddings import GPT4AllEmbeddings

embeddings_model = GPT4AllEmbeddings()

100%|██████████| 45.5M/45.5M [00:00<00:00, 80.6MiB/s]


Model downloaded at:  /root/.cache/gpt4all/ggml-all-MiniLM-L6-v2-f16.bin


In [ ]:
unwatched_movies['movies_names&genres'] = 'movie name : '+ unwatched_movies['title']+' & its genres :'+ unwatched_movies['genres']
unwatched_movies

,movieId,title,genres,movies_names&genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,movie name : Toy Story (1995) & its genres :Ad...
1,2,Jumanji (1995),Adventure|Children|Fantasy,movie name : Jumanji (1995) & its genres :Adve...
2,3,Grumpier Old Men (1995),Comedy|Romance,movie name : Grumpier Old Men (1995) & its gen...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,movie name : Waiting to Exhale (1995) & its ge...
4,5,Father of the Bride Part II (1995),Comedy,movie name : Father of the Bride Part II (1995...
...,...,...,...,...
62418,209157,We (2018),Drama,movie name : We (2018) & its genres :Drama
62419,209159,Window of the Soul (2001),Documentary,movie name : Window of the Soul (2001) & its g...
62420,209163,Bad Poems (2018),Comedy|Drama,movie name : Bad Poems (2018) & its genres :Co...
62421,209169,A Girl Thing (2001),(no genres listed),movie name : A Girl Thing (2001) & its genres ...


vector store

lets apply the Ebmbedding for the "unwatched_movies" and store the result in langchain vector store (Chroma)

Chroma has "similarity_search" method that takes a text and returns the most similar K(parameter) text to your input text.

So we will pass the 10 "watched_movies" and get the most similar 100 "unwatched_movies".


In [ ]:
! pip install -U chromadb

In [48]:
from langchain.vectorstores import Chroma

In [ ]:
 # load it into Chroma
db = Chroma.from_texts(unwatched_movies['movies_names&genres'], embeddings_model)


In [ ]:
random_movies['rating'] = random_movies['rating'].map(lambda x : str(x))

In [ ]:
random_movies['movies_names&genres&rating'] = 'movie name : '+ random_movies['title']+' & its genres :'+ random_movies['genres']+ ' & its rating is : '+random_movies['rating']
watched_movies_list = random_movies['movies_names&genres&rating']


In [ ]:
watched_movies_list[:]

59    movie name : Hour of the Wolf (Vargtimmen) (19...
58    movie name : Shrek 2 (2004) & its genres :Adve...
46    movie name : Good bye, Lenin! (2003) & its gen...
61    movie name : Swann in Love (Un amour de Swann)...
31    movie name : Piano Teacher, The (La pianiste) ...
52    movie name : Winter Light (Nattvardsgästerna) ...
50    movie name : Virgin Spring, The (Jungfrukällan...
13    movie name : Back to the Future Part III (1990...
39    movie name : Finding Nemo (2003) & its genres ...
12    movie name : Back to the Future Part II (1989)...
Name: movies_names&genres&rating, dtype: object

In [ ]:
unwatched_movies_list = []

for movie in random_movies['movies_names&genres&rating']:
  question = "what is the most similar movie for " + str(movie)
  most_similars = db.similarity_search(question,k=10) # k is the number of returned movies
  unwatched_movies_list.extend([text.page_content for text in most_similars])

print(unwatched_movies_list)

['movie_name : Toy Story (1995) its geners : Adventure|Animation|Children|Comedy|Fantasy', 'movie_name : Jumanji (1995) its geners : Adventure|Children|Fantasy', 'movie_name : Grumpier Old Men (1995) its geners : Comedy|Romance', 'movie_name : Waiting to Exhale (1995) its geners : Comedy|Drama|Romance', 'movie_name : Father of the Bride Part II (1995) its geners : Comedy', 'movie_name : Heat (1995) its geners : Action|Crime|Thriller', 'movie_name : Sabrina (1995) its geners : Comedy|Romance', 'movie_name : Tom and Huck (1995) its geners : Adventure|Children', 'movie_name : Sudden Death (1995) its geners : Action', 'movie_name : GoldenEye (1995) its geners : Action|Adventure|Thriller', 'movie_name : American President, The (1995) its geners : Comedy|Drama|Romance', 'movie_name : Dracula: Dead and Loving It (1995) its geners : Comedy|Horror', 'movie_name : Balto (1995) its geners : Adventure|Animation|Children', 'movie_name : Nixon (1995) its geners : Drama', 'movie_name : Cutthroat Isla

Now let's describe to our LLM his task in a prompt template.

it takes two variables the 10 "watched_movies" and the 100 "unwatched_movies".

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts.prompt import PromptTemplate

In [ ]:
template = """
you are a movie recommender agent. you will be provided with 10 watched movies by a user and 100 unwatched_movies.
the watched movies will be delimited with '###' and the unwatched movies will be delimited with ''$$$'.

your task is to recommend one movie from the unwatched_movies list for the user and give it a rate based on his ratings for the movies he had watched, and
explain why you recommend this movie for this user and why you gave it this rate.

The 10 watched movies have a name, genre, and rating.
the name is the name of the movie. The genre is the type of this movie it could be
one of these types (Action, Adventure, Animation, Children's, Comedy, Crime, Documentary, Drama, Fantasy,
Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western, no genres listed).
The user rated the watched movies on a scale of 1 to 5, where 1 indicates that he did not like the movie at all and 5 means that he loved the movie.

here is the watched movies list:
###
{watched_movies}
###

the 100 unwatched movies have a name and a genre also.
Here is th unwatched movies list:
$$$
{unwatched_movies}
$$$


"""

prompt = PromptTemplate(
    input_variables=["watched_movies", "unwatched_movies"],
    template=template
)



In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.generate([{
    'watched_movies': watched_movies_list,
    'unwatched_movies': unwatched_movies_list}])

In [ ]:
response

LLMResult(generations=[[Generation(text='Based on the watched and unwatched movies lists provided, I recommend "The Shawshank Redemption" for this user. This movie has a rating of 5 out of 5 from the user, indicating that he loved it very much. It is also an action genre which matches one of his favorite genres listed in the watched movies list.\nI rate this movie as 4/5 based on its overall quality and popularity among audiences.', generation_info=None)]], llm_output=None, run=[RunInfo(run_id=UUID('333941e5-e9ed-42c5-a286-71e455bd0efd'))])